
## ELT on Databricks
This is a simple ELT pipeline.
I first get the raw scraped data that's stored in an AWS S3 bucket, then filter out the posts with a "Site Meta" tag using a SQL query (since I don't care about having those in our visualisation). I then save the table in the Unity Catalog on Databricks to be used later on.

In [0]:
try:
    import dotenv
except:
    !pip install python-dotenv
import dotenv
from dotenv import load_dotenv
import os

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e566e957-e220-49cf-85cf-d1ff99e377f2/bin/python -m pip install --upgrade pip' command.


In [0]:
AWS_ACCESS = '<redacted>'
AWS_SECRET = '<redacted>'
access_key = os.getenv("AWS_ACCESS")
secret_key = os.getenv("AWS_SECRET")
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", AWS_ACCESS)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", AWS_SECRET)

aws_region = "eu-north-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")
file_path = 's3://lesswrong-data-feb-2025/lesswrong_posts_20250222_201800.json'

In [0]:
df = spark.read.option("multiline", "true").json(file_path)
df.head()

Out[3]: Row(posts=[Row(author_links=['/users/ruby?from=post_header', '/users/raemon?from=post_header', '/users/t3t?from=post_header', '/users/habryka4?from=post_header'], author_names=['Ruby', 'Raemon', 'RobertM', 'habryka'], post_karma='444', tags=['Site Meta', 'AI'], title='Welcome to LessWrong!', url='https://www.lesswrong.com/posts/bJ2haLkcGeLtTWaD5/welcome-to-lesswrong'), Row(author_links=['/users/joshc?from=post_header'], author_names=['joshc'], post_karma='344', tags=['Fiction', 'AI Risk', 'AI Risk Concrete Stories', 'AI Timelines', 'AI'], title='How AI Takeover Might Happen in 2 Years', url='https://www.lesswrong.com/posts/KFJ2LFogYqzfGB3uX/how-ai-takeover-might-happen-in-2-years'), Row(author_links=['/users/buck?from=post_header'], author_names=['Buck'], post_karma='124', tags=['History', 'Politics', 'Fact posts', 'Scholarship & Learning', 'World Modeling'], title='Some articles in “International Security” that I enjoyed', url='https://www.lesswrong.com/posts/MEfhRvpKPadJLTuTk

In [0]:
# Create a view or table
trends = "trends"
df.createOrReplaceTempView(trends)

In [0]:
%sql

select 
    post.title, 
    post.author_names, 
    post.tags as tags, 
    post.post_karma as karma
from trends 
LATERAL VIEW EXPLODE(posts) as post
WHERE NOT array_contains(post.tags, "Site Meta") 

title,author_names,tags,karma
How AI Takeover Might Happen in 2 Years,List(joshc),"List(Fiction, AI Risk, AI Risk Concrete Stories, AI Timelines, AI)",344
Some articles in “International Security” that I enjoyed,List(Buck),"List(History, Politics, Fact posts, Scholarship & Learning, World Modeling)",124
How to Make Superbabies,"List(GeneSmith, kman)","List(Biology, Human Genetics, Startups, World Optimization)",402


In [0]:
# persist the table 
_sqldf.write.mode("overwrite").saveAsTable("lesswrong_trends")
